In [4]:
config = {
    'host': 'localhost',
    'port' : 5432,
    'database': 'mimic_iv',
    'user': 'mimic_read_only_user',
    'password': 'mimiciv',
    'schema': 'hosp'
}

from sqlalchemy import create_engine, text
import pandas as pd



query = """
SELECT 
    p.*,
    CASE 
        WHEN matches[1] IS NOT NULL THEN CAST(matches[1] AS FLOAT)
        ELSE NULL
    END AS quantity,
    CASE 
        WHEN matches[1] IS NOT NULL THEN matches[3]
        ELSE NULL
    END AS unit,
    CASE 
        WHEN matches[1] IS NOT NULL THEN TRIM(BOTH ' ' FROM matches[5])
        ELSE p.prod_strength
    END AS form
FROM 
    hosp.prescriptions p,
    LATERAL REGEXP_MATCHES(p.prod_strength, '^([0-9]+(\.[0-9]+)?)?[ ]?([a-zA-Z]+(/[a-zA-Z]+)?)?([a-zA-Z ]+)$') AS matches
LIMIT 20000
    """





engine = create_engine(f'postgresql://{config["user"]}:{config["password"]}@{config["host"]}:{config["port"]}/{config["database"]}')
with engine.connect() as connection:
    df = pd.read_sql_query(text(query), connection)




<>:15: SyntaxWarning: invalid escape sequence '\.'
<>:15: SyntaxWarning: invalid escape sequence '\.'
C:\Users\jvoyer\AppData\Local\Temp\9\ipykernel_40772\1279218802.py:15: SyntaxWarning: invalid escape sequence '\.'
  query = """


In [2]:
df

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,order_provider_id,starttime,stoptime,drug_type,drug,...,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route,quantity,unit,form
0,16843859,22567390,66072452,16843859-951,951.0,P82RR6,2204-04-12 03:00:00,2204-04-18 16:00:00,MAIN,Polyethylene Glycol,...,None,17,g,1,PKT,NaN,PO/NG,17.0,g,Packet
1,16843859,22567390,68137571,16843859-958,958.0,P82RR6,2204-04-12 03:00:00,2204-04-12 12:00:00,MAIN,HYDROmorphone (Dilaudid),...,None,2-4,mg,1-2,TAB,NaN,PO/NG,2.0,mg,Tablet
2,16843859,22567390,72082526,16843859-950,950.0,P82RR6,2204-04-12 03:00:00,2204-04-18 16:00:00,MAIN,Senna,...,None,1,TAB,1,TAB,NaN,PO/NG,1.0,Table,t
3,16843859,22567390,80733151,16843859-954,954.0,P82RR6,2204-04-12 10:00:00,2204-04-18 16:00:00,MAIN,Sulfameth/Trimethoprim DS,...,None,1,TAB,1,TAB,1.0,PO/NG,NaN,None,DS Tablet
4,16843859,22567390,82563873,16843859-953,953.0,P82RR6,2204-04-12 13:00:00,2204-04-16 09:00:00,BASE,0.9% Sodium Chloride (Mini Bag Plus),...,None,100,mL,1,BAG,2.0,IV,100.0,mL,Bag
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,16861771,27622925,24023919,16861771-87,87.0,P92XOT,2147-03-16 10:00:00,2147-03-21 09:00:00,BASE,Iso-Osmotic Sodium Chloride,...,None,1,BAG,1,BAG,1.0,IV,50.0,ml,Bag
19996,16861771,27622925,24023919,16861771-87,87.0,P92XOT,2147-03-16 10:00:00,2147-03-21 09:00:00,MAIN,Famotidine,...,,20,mg,1,BAG,1.0,IV,20.0,mg,Premix Bag
19997,16861771,27622925,24404902,16861771-66,66.0,P04GOW,2147-03-15 02:00:00,2147-03-15 12:00:00,BASE,5% Dextrose,...,None,200,mL,200,mL,2.0,IV,1.0,Ba,g
19998,16861771,27622925,24404902,16861771-66,66.0,P04GOW,2147-03-15 02:00:00,2147-03-15 12:00:00,MAIN,Ciprofloxacin IV,...,,400,mg,1,BAG,2.0,IV,400.0,mg,Premix Bag
